In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
os.sys.path.append('environment/') #學習環境的py檔在environment的資料夾中

In [7]:
import os 
## turn back to main directory
os.chdir("../")
os.getcwd()

'/Users/apple/Git'

In [11]:
os.sys.path.append('/Users/apple/Git/ML-57/environment/') #學習環境的py檔在environment的資料夾中

# 簡化版的deep q learning： 僅把q-table換成類神經網路
＃在Reinforcement Learning中我們都利用Q_target（把它當作真實的Q值)來更新神經網路的weights。
公式： Q_target = Q(s) + alpha*( R(s,a) + Q(s_)*Gamma-Q(s) )  
(s_ 代表下一步的狀態，下一步的狀態有很多種可能，我們這裡選擇的s_是能得到最大Q的狀態，這種方法是比較agressive的方法，還有另外一種是SARSA有興趣可以自尋搜尋一下； alpha這邊我們設定為1）
因此公式就變成 Q_target = R(s,a) + max(Q(s_,a))*Gamma  

In [12]:
class QLearning:
    
    def __init__(
        self, 
        n_actions, #動作的維度，例如上下左右就有四維
        n_states, #用來描述狀態的維度，例如馬力歐在平面上就是二維
        gamma = 0.9, #遠見程度
        epsilon = 0.9,  #保守程度，越大就越容易用Q值大小來採取行動；越小則越容易產生隨機行動
        learning_rate = 0.001 #神經網路的更新率
    ):
    
        self.n_actions = n_actions
        self.n_states = n_states
        self.gamma = gamma
        self.epsilon = epsilon
        self.lr = learning_rate
        
        tf.reset_default_graph() ## 重新 build graph 需要跑這行
        self.sess = tf.Session() #宣告session
        #輸入current state
        self.state_input = tf.placeholder(shape = [None, self.n_states], 
                                          name = 'input',
                                          dtype = tf.float32)
        #q_target = R(s, action) + Q(s_)*Gamma 
        self.q_target = tf.placeholder(shape = [None, self.n_actions], 
                                       name = 'q_target',
                                       dtype = tf.float32)
        #搭建神經網路
        with tf.variable_scope('Q_table'):
            self.q_eval = self.build_network('net_eval') 
        
        # 管理神經網路的parameters
        self.Qnet_eval_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Q_table/net_eval')
        
        #計算q_target和q_eval的mse來更新神經網路的參數
        self.loss = tf.reduce_mean(tf.squared_difference(self.q_target, self.q_eval))
        self.train = tf.train.RMSPropOptimizer(self.lr).minimize(self.loss, var_list=self.Qnet_eval_params)
        
        self.sess.run(tf.global_variables_initializer()) #將神經網路初始化
        
            
    def build_network(self, scope): 
        with tf.variable_scope(scope):
            x_h1 = tf.layers.dense(inputs = self.state_input, units = 5, activation = tf.nn.tanh)        
            x_h2 = tf.layers.dense(inputs = x_h1, units = 5, activation = tf.nn.tanh)             
        return tf.layers.dense(inputs = x_h2, units = self.n_actions) #輸出‘不同動作’對應的Q值 
               
 
            
    def choose_action(self, current_state):
        if np.random.uniform() < self.epsilon: 
            #選擇產生估計Q值較大的行動
            q_eval = self.sess.run(self.q_eval, feed_dict={self.state_input: current_state[np.newaxis, :]})
            self.action = np.argmax(q_eval)
        else:
            #採取隨機行動
            self.action = np.random.randint(0, self.n_actions)
        return self.action
    
    def learn(self, current_state, reward, next_state):
        
        #算出實際q值並用此更新神經網路參數
        q_eval = self.sess.run(self.q_eval, feed_dict={self.state_input: current_state[np.newaxis, :]})
        q_eval_next = self.sess.run(self.q_eval, feed_dict={self.state_input: next_state[np.newaxis, :]})
        q_target = q_eval.copy()
        q_target[:, self.action] = reward + self.gamma*q_eval_next.max()
        _, self.cost = self.sess.run([self.train, self.loss], feed_dict={self.state_input: current_state[np.newaxis, :],
                                                                            self.q_target: q_target})
    def model_save(self, model_name):
        
        saver = tf.train.Saver()
        saver.save(self.sess, "saved_models/{}.ckpt".format(model_name))
    
    def model_restore(self, model_name):
        
        saver = tf.train.Saver()
        saver.restore(self.sess, "saved_models/{}.ckpt".format(model_name))
    

# 調整gamma並觀察學習情況

In [13]:
import Mario2
#建立環境==> env

#建立RL Agent==> RL

reward_record = [] #利用reward_record記錄每一回合的reward加總

for episode in range(50): #每次遊戲玩50回合
    
    total_reward = 0
    #initial environment並給出起始的state

    
    while True:
        #產生環境視窗如果使用hub則不需要  

   
               
        #當環境給出reward後累積此回合加總
        total_reward+= reward
        
        
        if done:
            #回合結束時記錄total_reward
            reward_record.append(total_reward)
            print('episode{}: total reward={}'.format(episode, total_reward))
            break
        #轉換到下一個state  

        
env.close()
            

NameError: name 'reward' is not defined

In [14]:
reward_result = pd.DataFrame(reward_record)
reward_result.plot()
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.show()

TypeError: Empty 'DataFrame': no numeric data to plot